# Phân tích giỏ hàng bằng thuật toán Apriori

## I. Giới thiệu 


## II. Dữ liệu

Đồ án sử dụng dataset có tên "The Instacart Online Grocery Shopping Dataset 2017", được đăng tải bởi chính Instacart trên Kaggle [1]. Dataset bao gồm hơn 3 triệu đơn hàng từ hơn 200.000 khách hàng đã đặt thông qua app và website của Instacart. 
Dataset gồm có 7 file dữ liệu riêng lẻ: 
1. aisles.csv (aisle_id, aisle)
2. departments.csv (department_id, department)
3. order_products_prior.csv (order_id, product_id, add_to_cart_order, reordered)
4. order_products_train.csv (order_id, product_id, add_to_cart_order, reordered)
5. orders.csv (order_id, user_id, eval_set, order_number, order_dow, order_hour_of_day, days_since_prior_order)
6. products.csv (product_id, product_name, aisles_id, department_id)
7. sample_submission.csv (order_id, products)

Trong phạm vi đồ án, có 3 file dữ liệu sẽ được sử dụng, liên kết với nhau là "order_products_train", "orders", và "products". 

## III. Thư viện python

In [35]:
# Những thư viện cần thiết 
import opendatasets as od # tải dữ liệu từ kaggle về jupyter notebook
import pandas as pd # xử lý dữ liệu.
import numpy as np # hỗ trợ tính toán. 
import matplotlib.pyplot as plt # trực quan hóa dữ liệu.
from mlxtend.preprocessing import TransactionEncoder # tiền xử lý dữ liệu để phân tích giỏ hàng.
from mlxtend.frequent_patterns import apriori, association_rules # thuật toán Apriori, và luật liên kết để phân tích giỏ hàng.

In [36]:
od.download('https://www.kaggle.com/c/instacart-market-basket-analysis/data')

Skipping, found downloaded files in ".\instacart-market-basket-analysis" (use force=True to force download)


Trích dẫn nguồn dữ liệu: https://www.kaggle.com/c/instacart-market-basket-analysis

## IV. Tiến hành phân tích

### 1. Khám phá dữ liệu - Exploring data in Python

In [46]:
# Đọc file dữ liệu thứ 1: order_products_train.
order_product = pd.read_csv(r'C:\Users\Oanh Le\Google Drive\My Drive\instacart-market-basket-analysis\order_products__prior.csv', usecols = col_prior)

# In DataFrame order_product.
order_product

MemoryError: Unable to allocate 1.00 MiB for an array with shape (131072,) and data type int64

File dữ liệu order_products_train gồm có order_id và product_id giúp liên kết với 2 bảng orders và products để tạo thành một DataFrame mới. 

In [42]:
# Đọc file dữ liệu thứ 2: orders. 
orders = pd.read_csv(r'C:\Users\Oanh Le\Google Drive\My Drive\instacart-market-basket-analysis\orders.csv')

# In DataFrame orders.
orders

MemoryError: Unable to allocate 1.00 MiB for an array with shape (131072,) and data type int64

File dữ liệu orders gồm 3.421.083 triệu đơn hàng đã được đặt. 

In [43]:
# Đọc file dữ liệu thứ 3: products. 
products = pd.read_csv(r'C:\Users\Oanh Le\Google Drive\My Drive\instacart-market-basket-analysis\products.csv')

# In DataFrame products.
products

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13
...,...,...,...,...
49683,49684,"Vodka, Triple Distilled, Twist of Vanilla",124,5
49684,49685,En Croute Roast Hazelnut Cranberry,42,1
49685,49686,Artisan Baguette,112,3
49686,49687,Smartblend Healthy Metabolism Dry Cat Food,41,8


File dữ liệu products gồm 49.688 sản phẩm đã được mua. 

In [ ]:
# INNER JOIN 3 DataFrame với nhau.
instacart_transaction = order_product.merge(orders, on = 'order_id').merge(products, on = 'product_id')

# Xuất file dữ liệu vừa INNER J"instacart_transaction.csv".
instacart_transaction.to_csv(r'C:\Users\Oanh Le\Google Drive\My Drive\instacart-market-basket-analysis\instacart_transaction.csv', index=False, header=True)

# In file vừa tạo. 
instacart_transaction

In [ ]:
# Tải file dữ liệu vừa mới xuất và chỉ lấy những cột cần thiết để chuẩn bị phân tích. 
# Chỉ lấy những cột sau.
col_insta_trans = ['order_id', 'product_id', 'user_id', 'product_name']

# Đọc file dữ liệu vừa mới xuất. 
instacart_transaction = pd.read_csv(r'C:\Users\Oanh Le\Google Drive\My Drive\instacart_datasets\instacart_transaction.csv', index_col=False, usecols=col_insta_trans)

# In DataFrame hoàn chỉnh. 
instacart_transaction

In [ ]:
# Thông tin sơ lược về DataFrame.
instacart_transaction.info()

In [ ]:
# Kiểm tra có trị null hay không 
instacart_transaction.isna().sum()

### 2. Initial exploratory analysis

In [ ]:
# Số lượng đơn hàng mà mỗi người dùng trong hơn 200.000 người đã đặt qua app và website Instacart 
order_count = instacart_transaction.groupby('user_id')['order_id'].count().reset_index(name='count')
order_count

In [ ]:
# Mỗi sản phẩm có bao nhiêu đơn đặt hàng
product_count = instacart_transaction.groupby('product_id', as_index=False)['order_id'].count()
product_count.rename(columns = {'order_id':'count'},inplace=True)
product_count

In [ ]:
# # Top 10 sản phẩm (product_name) được mua nhiều nhất. 
top10_count = instacart_transaction[['order_id','product_name']].groupby('product_name',as_index=False)['order_id'].count(
    ).sort_values(by='order_id',ascending=False)
top10_count.rename(columns = {'order_id':'count'},inplace=True)
top10_count.head(10)

In [ ]:
y_axis = top10_count['product_name'].head(10)
x_axis = top10_count['count'].head(10)

plt.barh(y_axis,x_axis, edgecolor='Silver')

plt.title('Top 10 sản phẩm được mua nhiều nhất', color='steelblue',
fontfamily='times new roman',fontsize=18)
plt.ylabel('Sản phẩm')
plt.xlabel('Số lượng')

plt.show()

In [ ]:
# Số lượng trung bình sản phẩm được bán ra.
purchase_count = instacart_transaction.groupby('order_id')['product_name'].count()
df = pd.DataFrame(purchase_count)
df.mean(axis=0)

### 3. Chuẩn bị dữ liệu để phân tích giỏ hàng

In [ ]:
# Bước đầu chuyển từ DataFrame sang transaction data để phân tích giỏ hàng 
# Đếm unique order IDs.
order_id = instacart_transaction['order_id'].unique()

# Tạo danh sách để chuyển DataFrame thành transaction data 
transaction = [list(instacart_transaction[instacart_transaction['order_id'] == u].product_name) for u in order_id]

"""
Danh sách dưới đây là gồm những đơn hàng (order_id) unique
Mỗi danh sách được ví như một giỏ hàng của khách hàng đã mua 
Và mỗi danh sách được phân cách bằng dấu []
"""
# In danh sách vừa tạo. 
transaction

In [ ]:
"""
Dưới đây là danh sách (giỏ hàng) gồm những sản phẩm (product_name) 
mà khách hàng hay mua cùng với nhau. 
"""
transaction[0] 

In [ ]:
"""
Tiếp tục danh sách vừa tạo sang dạng onehot encoding để tiến hành phân tích giỏ hàng 
Mỗi cột sẽ là một trong 49,653 sản phẩm (product_name)
Với giá trị False ~ (0): không xuất hiện trong giỏ hàng 
            True  ~ (1): có xuất hiện trong giỏ hàng 
            * Giỏ hàng ở đây chỉ 1 giỏ hàng của 1 khách hàng (unique).
"""
# Instantiate transaction encoder.
encoder = TransactionEncoder()
# One-hot encode instacart_transaction.
onehot = encoder.fit(transaction).transform(transaction)
# Sử dụng product_name là cột headers.
onehot = pd.DataFrame(onehot, columns = encoder.columns_)
# Print onehot header.
onehot

In [ ]:
onehot.mean()

In [ ]:
# Tính toán giá trị support. 
support_bananas_bags = np.logical_and(onehot['Banana'], onehot['Bag of Organic Bananas']).mean()
support_bananas = onehot['Bananas'].mean()
support_bags = onehot['Bag of Organic Bananas'].mean()
# Tính toán giá trị confidence and lift.
confidence = support_bananas_bags / support_bananas
lift = support_bananas_bags / (support_bananas * support_bags)
# In kết quả.
print(' Giá trị support cho sản phẩm "Bananas" là: %f\n' %support_bananas, 'Giá trị support của sản phẩm "Bag of Organic Bananas" là: %f\n' %support_bags, 'Giá trị confidence là: %f\n' %confidence, 'Giá trị lift là: %f\n' %lift)

Ý nghĩa của các chỉ số này: 

In [ ]:
# Tính toán chỉ số leverage. 
# Tính toán trước chỉ số support của Bananas và Limes.
support_bananas_limes = np.logical_and(onehot['Bananas'], onehot['Limes']).mean()
# Tính toán chỉ số của Bananas. 
support_bananas = onehot['Bananas'].mean()
# Tính toán chỉ số support của Limes. 
support_limes = onehot['Limes'].mean()
# Tính toán chỉ số leverage
leverage = support_bananas_limes - support_limes * support_bananas
print('Giá trị leverage là: %f' %leverage)

In [ ]:
# Tính toán chỉ số conviction. 
# Tính toán chỉ số support cho sản phẩm Bananas, Bags of Organic Bananas, và Limes. 
#support_bananas_limes = np.logical_and(onehot['Bananas'], onehot['Limes']).mean()
# Tính toán chỉ số support cho !Bags of Organic Bananas.
#support_not_bags = 1.0 - onehot['Bag of Organic Bananas'].mean()
# Tính toán chỉ số support của Bananas và !Bags of Organic Bananas
#support_bana_not_bag = support_bananas - support

In [ ]:
# Tính toán frequent itemsets
frequent_itemsets = apriori(onehot, use_colnames = True, min_support = 0.01)
# In tất cả số itemsets
print(len(frequent_itemsets))

In [ ]:
frequent_itemsets

# Pruning - loại bỏ những weak asscociation rules 

In [ ]:
# Tính toán association rules 
rules = association_rules(frequent_itemsets, metric = 'support', min_threshold = 0.0)
rules

In [ ]:
rules.columns

In [ ]:
rules[['antecedents', 'consequents']]

In [ ]:
# Lọc theo từng chỉ số khác 
#targeted_rules = rules[rules['consequents'] == {}]

In [ ]:
# Tính toán Zhang's rule
rules['zhang'] = zhangs_rule(rules)
# In rules mà là dissociation 
rules[rules['zhang'] < 0][['antecedents', 'consequents']]

# Tài liệu tham khảo
[1]. https://www.kaggle.com/c/instacart-market-basket-analysis
* https://pbpython.com/market-basket-analysis.html
* https://www.hackerearth.com/blog/developers/beginners-tutorial-apriori-algorithm-data-mining-r-implementation/
* https://www.datacamp.com/community/tutorials/market-basket-analysis-r
* https://www.kdnuggets.com/2019/12/market-basket-analysis.html
* https://www2.deloitte.com/cz/en/pages/deloitte-analytics/solutions/market-basket-analysis.html
* https://www.analyticsvidhya.com/blog/2021/10/end-to-end-introduction-to-market-basket-analysis-in-r/
* https://www.youtube.com/watch?v=aslTl6i-hpQ
* https://www.youtube.com/watch?v=guVvtZ7ZClw
* https://www.youtube.com/watch?v=SVM_pX0oTU8
* https://www.kaggle.com/pmarcelino/comprehensive-data-exploration-with-python
* https://ccc.inaoep.mx/~villasen/bib/Computing%20the%20minimum-support%20for%20mining%20frequent.pdf
* https://www.sciencedirect.com/science/article/pii/S0888613X04001392
* https://www.sciencedirect.com/science/article/pii/S2314728816300460
